In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,random_split
from torchvision.models import googlenet, resnet18, resnet50
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.transforms import functional as TF
import sklearn
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image

In [3]:
import random
import numpy as np
import torch

seed = 20236
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
RAVDESS = "test/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
CREMA = "test/cremad/AudioWAV/"
TESS = "test/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"
SAVEE = "test/surrey-audiovisual-expressed-emotion-savee/ALL/"

In [5]:
ravdess_dir_lis = os.listdir(RAVDESS)
path_list = []
gender_list = []
emotion_list = []

emotion_dic = {
    '03' : 'happy',
    '01' : 'neutral',
    '04' : 'sad',
    '05' : 'angry',
    '06' : 'fear',
    '07' : 'disgust',
}

for directory in ravdess_dir_lis:
    actor_files = os.listdir(os.path.join(RAVDESS, directory))
    for audio_file in actor_files: 
        part = audio_file.split('.')[0]
        key = part.split('-')[2]
        if key in emotion_dic:
            gender_code = int(part.split('-')[6])
            path_list.append(f"{RAVDESS}{directory}/{audio_file}")
            gender_list.append('female' if gender_code & 1 == 0 else 'male')
            emotion_list.append(emotion_dic[key])
            
ravdess_df = pd.concat([
    pd.DataFrame(path_list, columns=['path']),
    pd.DataFrame(gender_list, columns=['sex']),
    pd.DataFrame(emotion_list, columns=['emotion'])
], axis=1)

ravdess_df.head()

,path,sex,emotion
0,test/ravdess-emotional-speech-audio/audio_spee...,male,neutral
1,test/ravdess-emotional-speech-audio/audio_spee...,male,neutral
2,test/ravdess-emotional-speech-audio/audio_spee...,male,neutral
3,test/ravdess-emotional-speech-audio/audio_spee...,male,neutral
4,test/ravdess-emotional-speech-audio/audio_spee...,male,happy


In [6]:
crema_dir_list = os.listdir(CREMA)
path_list = []
gender_list = []
emotion_list = []

emotion_dic = {
    'HAP' : 'happy',
    'NEU' : 'neutral',
    'SAD' : 'sad',
    'ANG' : 'angry',
    'FEA' : 'fear',
    'DIS' : 'disgust',
}

female_id_list = [
    '1002', '1003', '1004', '1006', '1007', '1008', '1009', '1010', '1012', '1013', '1018', 
    '1020', '1021', '1024', '1025', '1028', '1029', '1030', '1037', '1043', '1046', '1047', 
    '1049', '1052', '1053', '1054', '1055', '1056', '1058', '1060', '1061', '1063', '1072', 
    '1073', '1074', '1075', '1076', '1078', '1079', '1082', '1084', '1089', '1091',
]

for audio_file in crema_dir_list:
    part = audio_file.split('_')
    key = part[2]
    if key in emotion_dic and part[3] == 'HI.wav':
        path_list.append(f"{CREMA}{audio_file}")
        gender_list.append('female' if part[0] in female_id_list else 'male')
        emotion_list.append(emotion_dic[key])

crema_df = pd.concat([
    pd.DataFrame(path_list, columns=['path']),
    pd.DataFrame(gender_list, columns=['sex']),
    pd.DataFrame(emotion_list, columns=['emotion'])
], axis=1)

crema_df.head()

,path,sex,emotion
0,test/cremad/AudioWAV/1001_IEO_ANG_HI.wav,male,angry
1,test/cremad/AudioWAV/1001_IEO_DIS_HI.wav,male,disgust
2,test/cremad/AudioWAV/1001_IEO_FEA_HI.wav,male,fear
3,test/cremad/AudioWAV/1001_IEO_HAP_HI.wav,male,happy
4,test/cremad/AudioWAV/1001_IEO_SAD_HI.wav,male,sad


In [7]:
tess_dir_list = os.listdir(TESS)
path_list = []
gender_list = []
emotion_list = [] 

emotion_dic = {
    'happy'   : 'happy',
    'neutral' : 'neutral',
    'sad'     : 'sad',
    'Sad'     : 'sad',
    'angry'   : 'angry',
    'fear'    : 'fear',
    'disgust'  : 'disgust',
}

for directory in tess_dir_list:
    audio_files = os.listdir(os.path.join(TESS, directory))
    for audio_file in audio_files:
        part = audio_file.split('.')[0]
        key = part.split('_')[2]
        if key in emotion_dic:
            path_list.append(f"{TESS}{directory}/{audio_file}") 
            gender_list.append('female') # female only dataset
            emotion_list.append(emotion_dic[key])
            
tess_df = pd.concat([
    pd.DataFrame(path_list, columns=['path']),
    pd.DataFrame(gender_list, columns=['sex']),
    pd.DataFrame(emotion_list, columns=['emotion'])
], axis=1)

tess_df.head()

,path,sex,emotion
0,test/toronto-emotional-speech-set-tess/tess to...,female,angry
1,test/toronto-emotional-speech-set-tess/tess to...,female,angry
2,test/toronto-emotional-speech-set-tess/tess to...,female,angry
3,test/toronto-emotional-speech-set-tess/tess to...,female,angry
4,test/toronto-emotional-speech-set-tess/tess to...,female,angry


In [8]:
savee_dir_list = os.listdir(SAVEE)
path_list = []
gender_list = []
emotion_list = []

emotion_dic = {
    'h'  : 'happy',
    'n'  : 'neutral',
    'sa' : 'sad',
    'a'  : 'angry',
    'f'  : 'fear',
    'd'  : 'disgust'
}

for audio_file in savee_dir_list:
    part = audio_file.split('_')[1]
    key = part[:-6]
    if key in emotion_dic:
        path_list.append(f"{SAVEE}{audio_file}")
        gender_list.append('male') # male only dataset
        emotion_list.append(emotion_dic[key])
        
savee_df = pd.concat([
    pd.DataFrame(path_list, columns=['path']),
    pd.DataFrame(gender_list, columns=['sex']),
    pd.DataFrame(emotion_list, columns=['emotion'])
], axis=1)

savee_df.head()

,path,sex,emotion
0,test/surrey-audiovisual-expressed-emotion-save...,male,angry
1,test/surrey-audiovisual-expressed-emotion-save...,male,angry
2,test/surrey-audiovisual-expressed-emotion-save...,male,angry
3,test/surrey-audiovisual-expressed-emotion-save...,male,angry
4,test/surrey-audiovisual-expressed-emotion-save...,male,angry


In [9]:
df = pd.concat([
    ravdess_df, 
    crema_df, 
    tess_df, 
    savee_df
], axis=0)
df.head()

,path,sex,emotion
0,test/ravdess-emotional-speech-audio/audio_spee...,male,neutral
1,test/ravdess-emotional-speech-audio/audio_spee...,male,neutral
2,test/ravdess-emotional-speech-audio/audio_spee...,male,neutral
3,test/ravdess-emotional-speech-audio/audio_spee...,male,neutral
4,test/ravdess-emotional-speech-audio/audio_spee...,male,happy


In [10]:
df.drop('sex', axis=1, inplace=True)
df.head()

,path,emotion
0,test/ravdess-emotional-speech-audio/audio_spee...,neutral
1,test/ravdess-emotional-speech-audio/audio_spee...,neutral
2,test/ravdess-emotional-speech-audio/audio_spee...,neutral
3,test/ravdess-emotional-speech-audio/audio_spee...,neutral
4,test/ravdess-emotional-speech-audio/audio_spee...,happy


In [11]:
import librosa
from pydub import AudioSegment, effects

In [12]:
def preprocess_audio(path):
    _, sr = librosa.load(path)
    raw_audio = AudioSegment.from_file(path)
    
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    trimmed, _ = librosa.effects.trim(samples, top_db=25)
    if (len(trimmed)>180000):
        padded = trimmed[:180000]
    else:
        padded = np.pad(trimmed, (0, 180000-len(trimmed)), 'constant')
    return padded, sr

In [12]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

In [13]:
emotion_dic = {
    'neutral' : 0,
    'happy'   : 1,
    'sad'     : 2, 
    'angry'   : 3, 
    'fear'    : 4, 
    'disgust' : 5
}

def encode(label):
    return emotion_dic.get(label)

In [14]:
list(emotion_dic.keys())

['neutral', 'happy', 'sad', 'angry', 'fear', 'disgust']

In [15]:
zcr_list = []
rms_list = []
mfccs_list = []
emotion_list = []

FRAME_LENGTH = 2048
HOP_LENGTH = 512

for row in df.itertuples(index=False):
    try: 
        y, sr = preprocess_audio(row.path)
        zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)
        zcr_list.append(zcr)
        rms_list.append(rms)
        mfccs_list.append(mfccs)

        emotion_list.append(encode(row.emotion))
    except:
        print(f"Failed for path: {row.path}")

In [16]:
X = np.concatenate((
    np.swapaxes(zcr_list, 1, 2), 
    np.swapaxes(rms_list, 1, 2), 
    np.swapaxes(mfccs_list, 1, 2)), 
    axis=2
)
X = X.astype('float32')

y = np.asarray(emotion_list)

In [28]:
df.iloc[1].path

'test/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-02-01.wav'

In [55]:
class EmotionDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.emotion_list = []

        self.FRAME_LENGTH = 2048
        self.HOP_LENGTH = 512

        for row in df.itertuples(index=False):
            self.emotion_list.append(encode(row.emotion))
            
        self.len = len(self.emotion_list)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        y, sr = preprocess_audio(df.iloc[idx].path)
        y = noise(y)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=self.HOP_LENGTH)
        mfccs = mfccs.T.astype('float32')
            
        label = self.emotion_list[idx]
        return mfccs, label

In [16]:
class EmotionDataset(Dataset):
    def __init__(self, images, labels):
        self.images = []
        self.emotions = torch.Tensor(labels).to(torch.int64)
        transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])

        for i in tqdm(range(len(images))):
            
            # MFCC를 이미지로 변환하여 크기를 조정합니다.
            image = Image.fromarray(images[i],"RGB")
            image = transform(image)
            self.images.append(image)
            
        self.len = len(images)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.emotions[idx]
        return image, label

In [17]:
class EmotionDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.emotions = torch.Tensor(labels).to(torch.int64)

    def __len__(self):
        return len(self.emotions)

    def __getitem__(self, idx):
        inputs = self.inputs[idx]
        labels = self.emotions[idx]
        return inputs, labels

In [38]:
class EmotionLSTM(nn.Module):
    def __init__(self, input_shape, hidden_size, n_class):
        super(EmotionLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_shape[1], hidden_size=hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_class)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = x[:, -1, :] 
        x = self.fc(x)
        return x

In [18]:
class EmotionLSTM(nn.Module):
    def __init__(self, input_shape, hidden_size, n_class):
        super(EmotionLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_shape[1], hidden_size=hidden_size, batch_first=True, dropout = 0.3)
        self.lstm2 = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True, dropout=0.3)
        self.lstm3 = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size//2, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size//2, n_class)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = x[:, -1, :] 
        x = self.fc(x)
        return x

In [56]:
dataset = EmotionDataset(df)
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

In [19]:
dataset = EmotionDataset(X,y)
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

In [20]:
dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [117]:
y, sr = preprocess_audio("../input.wav")
zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)

In [112]:
testX = np.concatenate((
    np.swapaxes(zcr, 0, 1), 
    np.swapaxes(rms, 0, 1), 
    np.swapaxes(mfccs, 0, 1)), 
    axis=1
)
testX = testX.astype('float32')

In [95]:
X.shape[1:3]

(352, 15)

In [61]:
model = EmotionLSTM(X.shape[1:3],256,6)

In [21]:
model = EmotionLSTM(X.shape[1:3],512,6)

In [38]:
model = EmotionLSTM((352, 13),512,6)

In [19]:
model = resnet18(pretrained=True)
num_features = model.fc.in_features
num_classes = 6
model.fc = nn.Linear(num_features, num_classes) 

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
len(X[0][0])

15

In [24]:
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy}%')
    return accuracy

In [72]:
from sklearn.metrics import f1_score

In [93]:
def f1score(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    predict = []
    label_data = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            label_data = label_data + labels.squeeze().tolist()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predict = predict + predicted.squeeze().tolist()

    accuracy = 100 * correct / total
    f1 = f1_score(label_data, predict,average="macro")
    print(f'Accuracy: {accuracy}%')
    print(f'f1 score: {f1*100}%')
    return accuracy

In [94]:
f1score(model,validation_loader)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 34.59it/s]

Accuracy: 81.25%
f1 score: 81.48568868514351%


81.25

In [25]:
num_epochs = 200

In [27]:
accuracy_list = []
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    accuracy_list.append(test(model,test_loader))
    torch.save(model.state_dict(), "result/model_LSTM_512_3_{}.pth".format(epoch))
    
print('Training finished!')

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 32.42it/s]


Test Accuracy: 35.18518518518518%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 30.84it/s]


Test Accuracy: 35.879629629629626%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 32.22it/s]


...

Test Accuracy: 79.39814814814815%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 32.48it/s]


Test Accuracy: 79.39814814814815%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 31.52it/s]

Test Accuracy: 78.00925925925925%
Training finished!


In [28]:
test(model, validation_loader)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 31.69it/s]

Test Accuracy: 78.70370370370371%


78.70370370370371

In [53]:
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_LSTM_512_{}.pth".format(epoch))
    
print('Training finished!')

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:02<00:00, 40.42it/s]


Test Accuracy: 42.129629629629626%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:02<00:00, 41.76it/s]


Test Accuracy: 40.0462962962963%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:02<00:00, 42.10it/s]


...

Test Accuracy: 74.07407407407408%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:02<00:00, 45.56it/s]


Test Accuracy: 78.00925925925925%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:02<00:00, 45.91it/s]

Test Accuracy: 77.08333333333333%
Training finished!


In [48]:
for epoch in range(60,60+num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_LSTM_256_{}.pth".format(epoch))
    
print('Training finished!')

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 69.62it/s]


Test Accuracy: 70.60185185185185%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 60.34it/s]


Test Accuracy: 72.45370370370371%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 67.83it/s]


...

Test Accuracy: 76.38888888888889%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 63.83it/s]


Test Accuracy: 78.47222222222223%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 60.82it/s]

Test Accuracy: 78.93518518518519%
Training finished!


In [105]:
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_resnet_new_{}.pth".format(epoch))
    
print('Training finished!')

100%|█████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 194.13it/s]


Test Accuracy: 71.47435897435898%


100%|█████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 186.43it/s]


Test Accuracy: 59.294871794871796%


100%|█████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 172.58it/s]


...

Test Accuracy: 79.16666666666667%


100%|█████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 168.45it/s]


Test Accuracy: 80.12820512820512%


Epoch [34/60]:  52%|███████████████████████████▏                        | 82/157 [00:03<00:03, 22.93it/s, loss=0.00913]


KeyboardInterrupt: 

## 남녀 데이터 둘다

In [106]:
df = pd.concat([
    ravdess_df, 
    crema_df, 
    tess_df, 
    savee_df
], axis=0)
df.head()

,path,sex,emotion
0,ravdess-emotional-speech-audio/audio_speech_ac...,male,neutral
1,ravdess-emotional-speech-audio/audio_speech_ac...,male,neutral
2,ravdess-emotional-speech-audio/audio_speech_ac...,male,neutral
3,ravdess-emotional-speech-audio/audio_speech_ac...,male,neutral
4,ravdess-emotional-speech-audio/audio_speech_ac...,male,happy


In [107]:
zcr_list = []
rms_list = []
mfccs_list = []
emotion_list = []

FRAME_LENGTH = 2048
HOP_LENGTH = 512

for row in df.itertuples(index=False):
    try: 
        y, sr = preprocess_audio(row.path)
        zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)
        zcr_list.append(zcr)
        rms_list.append(rms)
        mfccs_list.append(mfccs)

        emotion_list.append(encode(row.emotion))
    except:
        print(f"Failed for path: {row.path}")

Failed for path: ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_20/03-01-06-01-01-02-20.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_a13.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_a14.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_d01.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_d04.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_d08.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_d09.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_d10.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_d13.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_d14.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_d15.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_f11.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/DC_

Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_h15.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_n11.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_n14.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_sa04.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_sa05.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_sa10.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_sa11.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_sa12.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_sa13.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_sa14.wav
Failed for path: surrey-audiovisual-expressed-emotion-savee/ALL/KL_sa15.wav


In [108]:
X = np.concatenate((
    np.swapaxes(zcr_list, 1, 2), 
    np.swapaxes(rms_list, 1, 2), 
    np.swapaxes(mfccs_list, 1, 2)), 
    axis=2
)
X = X.astype('float32')

y = np.asarray(emotion_list)

In [109]:
dataset = EmotionDataset(X,y)
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

100%|█████████████████████████████████████████████████████████████████████████████| 4211/4211 [00:05<00:00, 749.53it/s]


In [110]:
model = resnet18(pretrained=True)
num_features = model.fc.in_features
num_classes = 6
model.fc = nn.Linear(num_features, num_classes) 

In [111]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [112]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [24]:
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_resnet_new_all_{}.pth".format(epoch))
    
print('Training finished!')

100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 177.73it/s]


Test Accuracy: 36.342592592592595%


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 166.05it/s]


Test Accuracy: 56.71296296296296%


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 178.52it/s]


...

Test Accuracy: 66.43518518518519%


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 171.95it/s]


Test Accuracy: 64.58333333333333%


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 167.70it/s]

Test Accuracy: 67.82407407407408%
Training finished!


100%|█████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 123.31it/s]

Test Accuracy: 80.76923076923077%


80.76923076923077

## googlenet

In [25]:
model = googlenet(pretrained=True)
num_features = model.fc.in_features
num_classes = 6
model.fc = nn.Linear(num_features, num_classes) 

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [27]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [28]:
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)                                                                          
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_googlenet_new_all_{}.pth".format(epoch))
    
print('Training finished!')

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 79.60it/s]


Test Accuracy: 19.675925925925927%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 77.06it/s]


Test Accuracy: 46.99074074074074%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 77.92it/s]


...

Test Accuracy: 70.37037037037037%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 81.58it/s]


Test Accuracy: 70.60185185185185%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 80.06it/s]

Test Accuracy: 66.89814814814815%
Training finished!


In [29]:
def preprocess_audio(path):
    _, sr = librosa.load(path)
    raw_audio = AudioSegment.from_file(path)
    
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    trimmed, _ = librosa.effects.trim(samples, top_db=25)
    if (len(trimmed)>240000):
        padded = trimmed[:240000]
    else:
        padded = np.pad(trimmed, (0, 240000-len(trimmed)), 'constant')
    return padded, sr

In [30]:
zcr_list = []
rms_list = []
mfccs_list = []
emotion_list = []

FRAME_LENGTH = 2048
HOP_LENGTH = 512

for row in df.itertuples(index=False):
    try: 
        y, sr = preprocess_audio(row.path)
        zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)
        zcr_list.append(zcr)
        rms_list.append(rms)
        mfccs_list.append(mfccs)

        emotion_list.append(encode(row.emotion))
    except:
        print(f"Failed for path: {row.path}")

In [31]:
X = np.concatenate((
    np.swapaxes(zcr_list, 1, 2), 
    np.swapaxes(rms_list, 1, 2), 
    np.swapaxes(mfccs_list, 1, 2)), 
    axis=2
)
X = X.astype('float32')

y = np.asarray(emotion_list)

In [32]:
dataset = EmotionDataset(X,y)
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

100%|█████████████████████████████████████████████████████████████████████████████| 4331/4331 [00:06<00:00, 652.57it/s]


In [33]:
model = resnet18(pretrained=True)
num_features = model.fc.in_features
num_classes = 6
model.fc = nn.Linear(num_features, num_classes) 

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [35]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [36]:
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)                                                                          
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_resnet_240000_all_{}.pth".format(epoch))
    
print('Training finished!')

100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 128.65it/s]


Test Accuracy: 28.47222222222222%


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 135.45it/s]


Test Accuracy: 54.398148148148145%


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 124.05it/s]


...

Test Accuracy: 67.5925925925926%


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 124.95it/s]


Test Accuracy: 67.82407407407408%


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 125.80it/s]

Test Accuracy: 65.50925925925925%
Training finished!


In [37]:
model = googlenet(pretrained=True)
num_features = model.fc.in_features
num_classes = 6
model.fc = nn.Linear(num_features, num_classes) 

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [39]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [40]:
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)                                                                          
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_googlenet_240000_all_{}.pth".format(epoch))
    
print('Training finished!')

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 54.02it/s]


Test Accuracy: 46.75925925925926%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 54.32it/s]


Test Accuracy: 50.925925925925924%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 56.93it/s]


...

Test Accuracy: 66.43518518518519%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:02<00:00, 53.82it/s]


Test Accuracy: 65.50925925925925%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:02<00:00, 53.71it/s]

Test Accuracy: 59.49074074074074%
Training finished!


In [45]:
df = pd.read_csv("datasets/emotion_test.csv")

In [46]:
df = df[["wav_id","상황"]]

In [58]:
from tqdm import tqdm

In [59]:
df.iloc[0][0]

'5f7940c59e04b149046cc19a'

In [ ]:
emotion_dic = {
    'neutral' : 0,
    'happy'   : 1,
    'sad'     : 2, 
    'angry'   : 3, 
    'fear'    : 4, 
    'disgust' : 5
}

In [90]:
emotion_dic = {'anger': 3, 'angry': 3, 'disgust': 5, 'fear': 4, 'happy': 1, 'happiness': 1,
                               'neutral': 0, 'sad': 2, 'sadness': 2, 'surprise': 6}

def encode(label):
    return emotion_dic.get(label)

In [91]:
import librosa
from pydub import AudioSegment, effects

In [92]:
def preprocess_audio(path):
    _, sr = librosa.load(path)
    raw_audio = AudioSegment.from_file(path)
    effects.normalize(raw_audio)
    
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    trimmed, _ = librosa.effects.trim(samples, top_db=25)
    if (len(trimmed)>180000):
        padded = trimmed[:180000]
    else:
        padded = np.pad(trimmed, (0, 180000-len(trimmed)), 'constant')
    return padded, sr

In [93]:
zcr_list = []
rms_list = []
mfccs_list = []
emotion_list = []

FRAME_LENGTH = 2048
HOP_LENGTH = 512

for row in df.itertuples(index=False):
    try: 
        y, sr = preprocess_audio(row.path)
        zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)
        zcr_list.append(zcr)
        rms_list.append(rms)
        mfccs_list.append(mfccs)

        emotion_list.append(encode(row.emotion))
    except:
        print(f"Failed for path: {row.path}")

In [77]:
zcr_list = []
rms_list = []
mfccs_list = []
emotion_list = []

FRAME_LENGTH = 2048
HOP_LENGTH = 512

for i in tqdm(range(len(df))):
    try:
        if (encode(df.iloc[i][1])==6): continue
        y, sr = preprocess_audio("datasets/emotion_audio_data/"+df.iloc[i][0]+".wav")
        zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)
        zcr_list.append(zcr)
        rms_list.append(rms)
        mfccs_list.append(mfccs)

        emotion_list.append(encode(df.iloc[i][1]))
    except:
        print(f"Failed for path: datasets/emotion_audio_data/{df.iloc[i][0]}.wav")

100%|██████████████████████████████████████████████████████████████████████████████| 8793/8793 [17:30<00:00,  8.37it/s]


In [94]:
X = np.concatenate((
    np.swapaxes(zcr_list, 1, 2), 
    np.swapaxes(rms_list, 1, 2), 
    np.swapaxes(mfccs_list, 1, 2)), 
    axis=2
)
X = X.astype('float32')

y = np.asarray(emotion_list)

In [26]:
tmpX = X.copy()
tmpy = y.copy()

In [27]:
class EmotionDataset(Dataset):
    def __init__(self, images, labels):
        self.images = []
        self.emotions = torch.Tensor(labels).to(torch.int64)
        transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])

        for i in tqdm(range(len(images))):
            
            # MFCC를 이미지로 변환하여 크기를 조정합니다.
            image = Image.fromarray(images[i],"RGB")
            image = transform(image)
            self.images.append(image)
            
        self.len = len(images)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.emotions[idx]
        return image, label

In [37]:
model = resnet50(pretrained=True)
num_features = model.fc.in_features
num_classes = 6
model.fc = nn.Linear(num_features, num_classes) 

In [38]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [30]:
dataset = EmotionDataset(X,y)
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

100%|█████████████████████████████████████████████████████████████████████████████| 4331/4331 [00:04<00:00, 870.71it/s]


In [31]:
dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [40]:
num_epochs = 100

In [95]:
dataset = EmotionDataset(X,y)
test_loader = DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True)

100%|█████████████████████████████████████████████████████████████████████████████| 4331/4331 [00:07<00:00, 607.47it/s]


In [41]:
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy}%')
    return accuracy

In [96]:
test(model,test_loader)

100%|██████████████████████████████████████████████████████████████████████████████| 1082/1082 [00:23<00:00, 45.16it/s]

Test Accuracy: 94.29297597042513%


94.29297597042513

In [80]:
test(model,test_loader)

100%|██████████████████████████████████████████████████████████████████████████████| 2106/2106 [00:48<00:00, 43.24it/s]

Test Accuracy: 21.616809116809115%


21.616809116809115

In [42]:
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)                                                                          
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_resnet50_final_{}.pth".format(epoch))
    
print('Training finished!')

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 85.83it/s]


Test Accuracy: 37.73148148148148%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 85.97it/s]


Test Accuracy: 24.76851851851852%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 84.75it/s]


...

Test Accuracy: 67.12962962962963%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 90.81it/s]


Test Accuracy: 66.20370370370371%


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 86.04it/s]


Test Accuracy: 68.05555555555556%
Training finished!


In [57]:
state = torch.load("result/model_resnet50_final_85.pth")

In [58]:
model = resnet50()
num_features = model.fc.in_features
num_classes = 6
model.fc = nn.Linear(num_features, num_classes) 
model.load_state_dict(state)

<All keys matched successfully>

In [59]:
model = model.to(device)

In [60]:
test(model,test_loader)

100%|██████████████████████████████████████████████████████████████████████████████| 2106/2106 [00:30<00:00, 68.51it/s]

Test Accuracy: 19.931149097815766%


19.931149097815766

In [71]:
from keras.models import Sequential
from keras import layers, optimizers, callbacks
import keras

In [72]:
MODEL = Sequential()
MODEL.add(layers.LSTM(256, return_sequences=True, input_shape=(X.shape[1:3])))
MODEL.add(layers.LSTM(256))
MODEL.add(layers.Dense(6, activation='softmax'))

print(MODEL.summary())

NotImplementedError: Cannot convert a symbolic Tensor (lstm_3/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [76]:
model = keras.models.load_model('.mdl_wts_256.hdf5')

NotImplementedError: Cannot convert a symbolic Tensor (lstm_6/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

## Test

In [8]:
import speech_recognition as speech

In [9]:
r = speech.Recognizer()
with speech.Microphone() as source:
    audio = r.listen(source)
    with open("input.wav","wb") as f:
        f.write(audio.get_wav_data())

In [55]:
emotion_decode = {
    0 : 'neutral',
    1 : 'happy',
    2 : 'sad', 
    3 : 'angry', 
    4 : 'fear', 
    5 : 'disgust'
}

In [10]:
import librosa
from pydub import AudioSegment, effects

In [25]:
def preprocess_audio(path):
    _, sr = librosa.load(path)
    raw_audio = AudioSegment.from_file(path)
    
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    trimmed, _ = librosa.effects.trim(samples, top_db=25)
    if (len(trimmed)>180000):
        padded = trimmed[:180000]
    else:
        padded = np.pad(trimmed, (0, 180000-len(trimmed)), 'constant')
    return padded, sr

In [12]:
input_wav = "input.wav"

FRAME_LENGTH = 2048
HOP_LENGTH = 512

try: 
    y, sr = preprocess_audio(input_wav)
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
    rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)
except:
    print(f"Failed for path: {input_wav}")
    
X = np.concatenate((
    np.swapaxes(zcr, 0, 1), 
    np.swapaxes(rms, 0, 1), 
    np.swapaxes(mfcc, 0, 1)), 
    axis=1
)
X = X.astype('float32')

Failed for path: input.wav


In [3]:
input_wav = "input.wav"

FRAME_LENGTH = 2048
HOP_LENGTH = 512

try: 
    y, sr = preprocess_audio(input_wav)
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
    rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)
except:
    print(f"Failed for path: {input_wav}")
    
X = np.concatenate((
    np.swapaxes(zcr, 0, 1), 
    np.swapaxes(rms, 0, 1), 
    np.swapaxes(mfcc, 0, 1)), 
    axis=1
)
X = X.astype('float32')

In [4]:
from emotion_model import emotion_predict

In [5]:
result = emotion_predict(X)

In [6]:
print(result)

neutral


In [57]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

image = Image.fromarray(X,"RGB")
image = transform(image)
image = image.unsqueeze(dim=0)

In [49]:
state = torch.load("test/result/model_resnet_new_all_55.pth")

In [50]:
model = resnet18()
num_features = model.fc.in_features
num_classes = 6
model.fc = nn.Linear(num_features, num_classes) 
model.load_state_dict(state)

<All keys matched successfully>

In [58]:
model.eval()
result = model.forward(image)

In [59]:
emotion_decode[torch.argmax(result.squeeze()).item()]

'neutral'

In [61]:
import nemo.collections.asr as nemo_asr
asr_net = nemo_asr.models.ASRModel.from_pretrained("eesungkim/stt_kr_conformer_transducer_large")

[NeMo W 2023-06-13 00:33:58 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-06-13 00:33:59 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[NeMo I 2023-06-13 00:34:01 mixins:170] Tokenizer SentencePieceTokenizer initialized with 2261 tokens


[NeMo W 2023-06-13 00:34:02 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/public/rw/chris/manifests/ksponspeech//train.json
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: true
    trim_silence: false
    max_duration: 17
    min_duration: 0.1
    shuffle_n: 2048
    
[NeMo W 2023-06-13 00:34:02 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data/public/rw/chris/manifests/ksponspeech//dev.json
    sample_rate: 16000
    batch_size: 16
    shuffle: false
    num_workers: 8
    pin_memory: true
    use_start_e

[NeMo I 2023-06-13 00:34:02 features:287] PADDING: 0
[NeMo I 2023-06-13 00:34:03 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': 5}
[NeMo I 2023-06-13 00:34:03 audio_preprocessing:517] Numba CUDA SpecAugment kernel is being used
[NeMo I 2023-06-13 00:34:04 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from C:\Users\good\.cache\huggingface\hub\models--eesungkim--stt_kr_conformer_transducer_large\snapshots\fdc8412fe0d089913524767b20ff244ff1007ed0\stt_kr_conformer_transducer_large.nemo.


In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
asr_net = asr_net.to(device)

In [64]:
asr_net.transcribe(['input.wav'])

(['이엇 허'], ['이엇 허'])